In [5]:
import sys
import os
import tensorflow as tf
import numpy as np
import seaborn as sns
sys.path.append("/".join(os.getcwd().split("/")[:-1]))

In [6]:
from src.utils.tf_helper import _soft_f1_macro, f1_scores_1, f1_scores_2, f1_scores_3
from src.utils.audio import load_processed_audio
from src.utils.misc_utils import transform_labels
from src.make_dataset import dataset_input_fn
from src.models import nn
from src.settings.general import *

In [7]:
tf.enable_eager_execution()

positives, negatives, backgrounds = load_processed_audio()

In [ ]:
dataset = dataset_input_fn(positives, negatives, backgrounds, NEGATIVE_RATIO, BATCH_SIZE)

latest = tf.train.latest_checkpoint("../logs/checkpoints")

model = tf.keras.models.load_model('{}/model.h5'.format(LOG_DIR), custom_objects={'_soft_f1_macro': _soft_f1_macro,
                                                                                      'f1_scores_1': f1_scores_1,
                                                                                      'f1_scores_2': f1_scores_2,
                                                                                      'f1_scores_3': f1_scores_3})

model.load_weights(latest)

In [8]:
from src.make_dataset import add_word, background_generator

In [11]:
dataset = (tf.data.Dataset.from_generator(background_generator(backgrounds,
                                                                int(BACKGROUND_DURATION_MS*FRAME_RATE/1000),
                                                                32),
                                           output_types=tf.float32)
           .map(lambda backgrounds: tf.py_function(add_word(positives, negatives, .3), [backgrounds], (tf.float32, tf.int32))))
                                                                                        
                                                                                        

In [12]:
for x in dataset.take(1):
    pass

UnknownError: InvalidArgumentError: cannot compute Pack as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Pack] name: 0
Traceback (most recent call last):

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 207, in __call__
    return func(device, token, args)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 109, in __call__
    ret = self._func(*args)

  File "/Users/az01640/Projets/multrigger-word/src/make_dataset.py", line 59, in fn_batch
    return tf.stack(result)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/util/dispatch.py", line 180, in wrapper
    return target(*args, **kwargs)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py", line 1035, in stack
    return ops.convert_to_ten

In [11]:
import pydub
from pydub.playback import play

i = 3
audio = x[0][i].numpy()
play(pydub.AudioSegment(audio.tobytes(), 
                   frame_rate=FRAME_RATE, channels=1, 
                   sample_width=audio.dtype.itemsize))

KeyboardInterrupt: 

In [ ]:
sns.relplot(x='x', y='y', row='label', data=transform_labels(x[1][i].numpy(), LABEL_MAP_DICT), 
            kind='line', height=2, aspect=2)

In [8]:
sns.relplot(x='x', y='y', row='label', data=transform_labels(predictions, LABEL_MAP_DICT), 
            kind='line', height=2, aspect=2)

NameError: name 'predictions' is not defined

In [70]:
import os
import time
import numpy as np

import tensorflow as tf

backgrounds = np.eye(10)
WORDS = list(range(2, 10))

def insert_random(background):
    
    backs = []

    for i in range(int(tf.shape(background)[0])):
        word = np.random.choice(WORDS)
        back = background[i]
        backs.append([tf.concat([back[:2], [word], back[3:]], axis=0), np.array([1], dtype=np.int32)])

    print(backs)
    
    return backs

ds = (tf.data.Dataset.from_tensor_slices(backgrounds)
      .batch(5)
      .map(lambda x: tf.py_function(insert_random, [x], [tf.float64, tf.int32])))

for record in ds:
    print(record)

[[<tf.Tensor: id=13244, shape=(10,), dtype=float64, numpy=array([1., 0., 7., 0., 0., 0., 0., 0., 0., 0.])>, array([1], dtype=int32)], [<tf.Tensor: id=13259, shape=(10,), dtype=float64, numpy=array([0., 1., 8., 0., 0., 0., 0., 0., 0., 0.])>, array([1], dtype=int32)], [<tf.Tensor: id=13274, shape=(10,), dtype=float64, numpy=array([0., 0., 3., 0., 0., 0., 0., 0., 0., 0.])>, array([1], dtype=int32)], [<tf.Tensor: id=13289, shape=(10,), dtype=float64, numpy=array([0., 0., 9., 1., 0., 0., 0., 0., 0., 0.])>, array([1], dtype=int32)], [<tf.Tensor: id=13304, shape=(10,), dtype=float64, numpy=array([0., 0., 5., 0., 1., 0., 0., 0., 0., 0.])>, array([1], dtype=int32)]]


UnknownError: InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [10] != values[1].shape = [1] [Op:Pack] name: packed/
Traceback (most recent call last):

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 207, in __call__
    return func(device, token, args)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 115, in __call__
    for (x, dtype) in zip(ret, self._out_dtypes)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 115, in <listcomp>
    for (x, dtype) in zip(ret, self._out_dtypes)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 101, in _convert
    return ops.convert_to_tensor(value, dtype=dtype)

  File "/Users/az01640/Projets/multrigger-word/.venv/lib/python3.

In [21]:
fn_batch(list(elems))

NameError: name 'fn_batch' is not defined

In [57]:
from src.make_dataset import *

In [78]:
ds = tf.data.Dataset.from_generator(elems, output_types=tf.int32)

TypeError: `generator` must be callable.

In [73]:
for x in ds:
    pass

In [74]:
x

<tf.Tensor: id=9680, shape=(2, 3), dtype=int64, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>